In [1]:
from utils import create_movie_graph_db, query_db
from neo4j import GraphDatabase

/Users/aappopulkkinen/repos/llm-finetuning-public/utils.py:19: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  labels.append(0 if label_dir is "neg" else 1)


### Generate a simple movie, actor, director graph database

In [2]:
create_movie_graph_db()

### Test Cypher queries

In [3]:
# Match all nodes
res = query_db("""
  MATCH (n) 
  RETURN count(n) AS numberOfNodes
  """)
print(res)

[{'numberOfNodes': 27770}]


In [4]:
# Match only movie nodes
res = query_db("""
  MATCH (m:Movie) 
  RETURN count(m) AS numberOfMovies
  """)
print(res)

[{'numberOfMovies': 8964}]


In [5]:
# Match all relationships
res = query_db("""
  MATCH ()<-[r]-()
  RETURN count(r) AS numberOfRelationships
  """)
print(res)

[{'numberOfRelationships': 65326}]


In [6]:
# Get all relationship types
res = query_db("""
  MATCH ()<-[r]-()
  RETURN DISTINCT TYPE(r) as relationshipType
  """)
print(res)

[{'relationshipType': 'DIRECTED'}, {'relationshipType': 'ACTED_IN'}, {'relationshipType': 'IN_GENRE'}]


In [7]:
# Match only ACTED_IN relationships
res = query_db("""
  MATCH ()<-[a:ACTED_IN]-()
  RETURN count(a) AS numberOfActedInRelationships
  """)
print(res)

[{'numberOfActedInRelationships': 35494}]


In [8]:
# Match only movies with Pierce Brosnan acting in them
res = query_db("""
  MATCH (m)<-[a:ACTED_IN]-(p:Person {name: "Pierce Brosnan"})
  RETURN m.title as brosnanMovie
  """)
print(res)

[{'brosnanMovie': 'How to Make Love Like an Englishman'}, {'brosnanMovie': 'Salvation Boulevard'}, {'brosnanMovie': "I Don't Know How She Does It"}, {'brosnanMovie': 'Ghost Writer, The'}, {'brosnanMovie': 'Oceans (Océans)'}, {'brosnanMovie': 'Mamma Mia!'}, {'brosnanMovie': 'Butterfly on a Wheel (Shattered)'}, {'brosnanMovie': 'Matador, The'}, {'brosnanMovie': 'Laws of Attraction'}, {'brosnanMovie': 'After the Sunset'}, {'brosnanMovie': 'Nomads'}, {'brosnanMovie': 'Die Another Day'}, {'brosnanMovie': 'Evelyn'}, {'brosnanMovie': 'Tailor of Panama, The'}, {'brosnanMovie': 'World Is Not Enough, The'}, {'brosnanMovie': 'Thomas Crown Affair, The'}, {'brosnanMovie': 'Mars Attacks!'}, {'brosnanMovie': 'Tomorrow Never Dies'}, {'brosnanMovie': "Dante's Peak"}, {'brosnanMovie': 'GoldenEye'}, {'brosnanMovie': 'Lawnmower Man, The'}, {'brosnanMovie': 'Mrs. Doubtfire'}]


In [9]:
# Match only movies with Pierce Brosnan acting in them
res = query_db("""
  MATCH (d:Person)-[:DIRECTED]->(m)<-[a:ACTED_IN]-(p:Person {name: "Pierce Brosnan"})
  RETURN d.name as brosnanMovieDirector
  """)
print(res)

[{'brosnanMovieDirector': 'Tom Vaughan'}, {'brosnanMovieDirector': 'George Ratliff'}, {'brosnanMovieDirector': 'Douglas McGrath'}, {'brosnanMovieDirector': 'Roman Polanski'}, {'brosnanMovieDirector': 'Jacques Cluzaud'}, {'brosnanMovieDirector': 'Jacques Perrin'}, {'brosnanMovieDirector': 'Phyllida Lloyd'}, {'brosnanMovieDirector': 'Mike Barker'}, {'brosnanMovieDirector': 'Richard Shepard'}, {'brosnanMovieDirector': 'Peter Howitt'}, {'brosnanMovieDirector': 'Brett Ratner'}, {'brosnanMovieDirector': 'John McTiernan'}, {'brosnanMovieDirector': 'Lee Tamahori'}, {'brosnanMovieDirector': 'Bruce Beresford'}, {'brosnanMovieDirector': 'John Boorman'}, {'brosnanMovieDirector': 'Michael Apted'}, {'brosnanMovieDirector': 'John McTiernan'}, {'brosnanMovieDirector': 'Tim Burton'}, {'brosnanMovieDirector': 'Roger Spottiswoode'}, {'brosnanMovieDirector': 'Roger Donaldson'}, {'brosnanMovieDirector': 'Martin Campbell'}, {'brosnanMovieDirector': 'Brett Leonard'}, {'brosnanMovieDirector': 'Chris Columbus'

In [10]:
# Fetch movies that have some actor in common with GoldenEye
res = query_db("""
    MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)-[:ACTED_IN]->(goldenEye:Movie {title: 'GoldenEye'})
    RETURN m.title as goldenEyeActorMovie
    """)
print(res)

[{'goldenEyeActorMovie': 'How to Make Love Like an Englishman'}, {'goldenEyeActorMovie': 'Salvation Boulevard'}, {'goldenEyeActorMovie': "I Don't Know How She Does It"}, {'goldenEyeActorMovie': 'Ghost Writer, The'}, {'goldenEyeActorMovie': 'Oceans (Océans)'}, {'goldenEyeActorMovie': 'Mamma Mia!'}, {'goldenEyeActorMovie': 'Butterfly on a Wheel (Shattered)'}, {'goldenEyeActorMovie': 'Matador, The'}, {'goldenEyeActorMovie': 'Laws of Attraction'}, {'goldenEyeActorMovie': 'After the Sunset'}, {'goldenEyeActorMovie': 'Nomads'}, {'goldenEyeActorMovie': 'Die Another Day'}, {'goldenEyeActorMovie': 'Evelyn'}, {'goldenEyeActorMovie': 'Tailor of Panama, The'}, {'goldenEyeActorMovie': 'World Is Not Enough, The'}, {'goldenEyeActorMovie': 'Thomas Crown Affair, The'}, {'goldenEyeActorMovie': 'Mars Attacks!'}, {'goldenEyeActorMovie': 'Tomorrow Never Dies'}, {'goldenEyeActorMovie': "Dante's Peak"}, {'goldenEyeActorMovie': 'Lawnmower Man, The'}, {'goldenEyeActorMovie': 'Mrs. Doubtfire'}, {'goldenEyeActor